In [47]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import pandas as pd 
from torch.utils.data import Dataset, DataLoader
from time import perf_counter 
from torch.autograd import Variable
import torch.optim as optim
import math

In [48]:
dfRaw = pd.read_csv("macro40.csv") 
### "Data preprocessing
dfP = dfRaw.drop("Date",axis=1) # only in the first time
df = dfP.sub(dfP.mean(axis=0), axis=1).div(dfP.std(0),axis=1) #standardization

In [49]:
# Hyper Parameters
p = 4
N = 40
Smp_size = 100
Pred_size = len(df.index)-Smp_size-p

batch_size = 1
n_steps = p ###Key understanding here!
n_inputs = N
n_neurons = 1 #adjustable
n_outputs = N
# the layer is set to be 1 and fixed

In [50]:
class RNNModel(nn.Module):
    def __init__(self, batch_size, n_steps, n_inputs, n_neurons, n_outputs):
        super(RNNModel, self).__init__()
        
        self.n_neurons = n_neurons
        self.batch_size = batch_size
        self.n_steps = n_steps
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        
        self.basic_rnn = nn.RNN(self.n_inputs, self.n_neurons) 
        
        self.FC = nn.Linear(self.n_neurons, self.n_outputs)
        
    def init_hidden(self,):
        # (num_layers, batch_size, n_neurons)
        return (torch.zeros(1, self.batch_size, self.n_neurons))
        
    def forward(self, X):
        # transforms X to dimensions: n_steps X batch_size X n_inputs
        X = X.permute(1, 0, 2) 
        
        self.batch_size = X.size(1)
        self.hidden = self.init_hidden()
        
        lstm_out, self.hidden = self.basic_rnn(X, self.hidden)      
        out = self.FC(self.hidden)
        
        return out.view(-1, self.n_outputs) # batch_size X n_output

In [51]:
### "Generate Linear Time Series Inputs and Targetted Output
# return large transition Matrix A
def kronecker(A, B):
    return torch.ger(A.view(-1), B.view(-1)).reshape(*(A.size() + B.size())).permute([0, 2, 1, 3]).reshape(A.size(0)*B.size(0),A.size(1)*B.size(1))

### " F-norm of A
class L2LossFun(nn.Module):
    
    def __init__(self):
        super(L2LossFun, self).__init__()
    def forward(self, A_Est, A_True):
        gap = math.sqrt(torch.sum((A_Est - A_True)**2))
        return gap

class LinfLossFun(nn.Module):
    
    def __init__(self):
        super(LinfLossFun, self).__init__()
    def forward(self, A_Est, A_True):
        gap = max(abs(torch.squeeze(A_Est-A_True))).item()
        return gap

In [52]:
class RealDataset(Dataset):
    
    def __init__(self, p, N, Smp_size, df):
        self.X = []
        self.y = []
        for i in range(Smp_size):
            if i == 0:
                input_TS = torch.tensor(df.iloc[ [3,2,1,0] , : ].values).view(1,1,p,N) # need to adjust with different p
                self.X.append(torch.squeeze(input_TS.view(1,1,p,N), dim = 0))
                output_TS = torch.squeeze(torch.tensor(df.iloc[ p:(p+1) , : ].values).permute(1,0).view(1,1,N,1))
                self.y.append(output_TS[:N])  
            else:
                input_TS = torch.cat([self.y[i-1].view(1,1,1,N), input_TS], dim = 2)
                self.X.append(torch.squeeze(input_TS[:,:,:p,:N], dim = 0))
                out_tmp = torch.squeeze(torch.tensor(df.iloc[ (i+p):(i+p+1) , : ].values).permute(1,0).view(1,1,N,1))
                self.y.append(out_tmp[:N])
                
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
    def __len__(self):
        return len(self.X)

In [56]:
# input to GPU
device = torch.device("cuda:2")
device2 = torch.device('cpu')

In [57]:
names = {}
names["predErrorRNN1"] =  []
names["predErrorRNN1Linf"] =  []
names["LTRRNN1_pred"] = []

In [58]:
distance = L2LossFun()
distanceLinf = LinfLossFun()

t1_start = perf_counter() 

for k in range(Pred_size):
    
    if k == 0:
        dsT = RealDataset(p=p, N=N, Smp_size=Smp_size+k, df=df)
    else:
        dsT = dsF
        
    dsF = RealDataset(p=p, N=N, Smp_size=Smp_size+k+1, df=df)
    X_F,y_F = dsF[Smp_size+k]
    X_F = X_F.view(1,p,N)
    ds = DataLoader(dsT, batch_size=Smp_size+k, shuffle=False)

    RNNmodel = RNNModel(batch_size, n_steps, n_inputs, n_neurons, n_outputs)

    criterion = nn.MSELoss()
    optimizer = optim.SGD(RNNmodel.parameters(), lr = 0.01, momentum=0.9)

    loss_last = 1000
    loss_new = 0
    
    i = 0
    while abs(loss_last - loss_new) > 0.00000001:
        if i > 0:
            loss_last = loss_new  
        for ix, (_x, _y) in enumerate(ds):

            #=========make inpur differentiable=======================
            _x = Variable(_x.view(-1,p,N)).float()
            _y = torch.squeeze(Variable(_y).float())
            #========forward pass=====================================
            yhat = RNNmodel(_x).float()
            loss = criterion(yhat, _y)
            #=======backward pass=====================================
            optimizer.zero_grad() 
            loss.backward() 
            optimizer.step() 
            loss_new = loss.item()
        i = i + 1
        
    X_F = Variable(X_F).float()
    y_F = torch.tensor(Variable(y_F).float())
    y_pred = RNNmodel(X_F).float()
    names["LTRRNN1_pred"].append(y_pred)
    predError = distance(y_pred, y_F)
    names["predErrorRNN1"].append(predError)
    print("PredError is {}.".format(predError))
    predErrorLinf = distanceLinf(y_pred, y_F)
    names["predErrorRNN1Linf"].append(predErrorLinf)
    print("PredErrorLinf is {}.".format(predErrorLinf))
        
    print(k)

t1_stop = perf_counter() 
print("Elapsed time during the whole program in seconds:", 
                                        t1_stop-t1_start) 
Real_100RNN1s = names
torch.save(Real_100RNN1s, "Real_100RNN1s.py")

/Users/amy/.virtualenvs/dl4cv/lib/python3.6/site-packages/ipykernel_launcher.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


PredError is 6.1136631270503194.
PredErrorLinf is 2.060002326965332.
0
PredError is 7.378202874347472.
PredErrorLinf is 2.7261533737182617.
1
PredError is 7.0090309198072775.
PredErrorLinf is 2.765146255493164.
2
PredError is 7.730295542684386.
PredErrorLinf is 2.5243568420410156.
3
PredError is 6.5646274569606495.
PredErrorLinf is 2.354713201522827.
4
PredError is 4.735772559233402.
PredErrorLinf is 1.5194296836853027.
5
PredError is 7.288058654415132.
PredErrorLinf is 2.6546294689178467.
6
PredError is 12.507723444014626.
PredErrorLinf is 3.37967848777771.
7
PredError is 5.152084980352653.
PredErrorLinf is 1.7725173234939575.
8
PredError is 6.747889153366632.
PredErrorLinf is 4.31044340133667.
9
PredError is 4.876353931703105.
PredErrorLinf is 1.9481483697891235.
10
PredError is 3.9764042131439044.
PredErrorLinf is 1.7377949953079224.
11
PredError is 4.6020596661469275.
PredErrorLinf is 2.2098724842071533.
12
PredError is 3.3152245886487317.
PredErrorLinf is 1.451372742652893.
13
Pre

In [59]:
from statistics import mean,median
print(mean(names["predErrorRNN1"]))
print(mean(names["predErrorRNN1Linf"]))

5.57505631112363
2.233087197939555


In [ ]:
print(mean(names["predError_smp"]))